In [1]:
#Import statements
import geopandas
import pandas as pd
import requests
from shapely.geometry import Point

In [2]:
#Make API request
request_obj = requests.get("https://phl.carto.com/api/v2/sql?q=SELECT * FROM incidents_part1_part2 WHERE dispatch_date_time >= '2011-01-01' AND dispatch_date_time < '2016-01-01'")
request_json = request_obj.json()

In [3]:
#Convert rows (in dict) to df
df = pd.DataFrame.from_dict(request_json['rows'])

#Drop rows missing location data
df = df.dropna(subset=["point_x", "point_y"])

#Reshape to group by year and crimes for each geo
df['year'] = pd.to_datetime(df['dispatch_date_time']).dt.year
grouped_df = df.groupby(['year','point_x','point_y']).size().reset_index(name='crime_count')

#Convert to geodf

grouped_df['geometry'] = [Point(xy) for xy in zip(grouped_df.point_x, grouped_df.point_y)]
grouped_df.head(1)

,year,point_x,point_y,crime_count,geometry
0,2011,-75.275694,39.977272,1,POINT (-75.27569444 39.97727246)


In [4]:
crime_geo = geopandas.GeoDataFrame(grouped_df, geometry='geometry')
crime_geo.head(1)

,year,point_x,point_y,crime_count,geometry
0,2011,-75.275694,39.977272,1,POINT (-75.27569444 39.97727246)


In [5]:
#load the evictions data (from geojson) into a geodf
evictions = geopandas.read_file('data/block-groups.geojson')[['GEOID','geometry']]


In [6]:
#Merge the geodfs
eviction_merge = geopandas.sjoin(crime_geo, evictions, how="inner", op='intersects')

/anaconda3/envs/exe/lib/python3.7/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')


In [7]:
eviction_merge.head(1)

,year,point_x,point_y,crime_count,geometry,index_right,GEOID
0,2011,-75.275694,39.977272,1,POINT (-75.27569444 39.97727246),7939,421010098022


In [8]:
#Reshape to get year- bloc_id - count of crimes
eviction_grouped = eviction_merge.groupby(['year','GEOID'])['crime_count'].sum().reset_index(name='crime_count')

In [9]:
eviction_grouped

,year,GEOID,crime_count
0,2011,420171014012,1
1,2011,420454021001,4
2,2011,420454023001,4
3,2011,420912001041,2
4,2011,420912001051,3
5,2011,420912019021,1
6,2011,420912020002,1
7,2011,420912022011,1
8,2011,420912022013,1
9,2011,420912022014,21
